In [34]:
import pm4py
import pandas as pd

# Data pre-processing

In [35]:
event_log = pm4py.read_xes('data/end_A_event_log.xes')

parsing log, completed traces ::   0%|          | 0/17549 [00:00<?, ?it/s]

In [36]:
df = pm4py.convert_to_dataframe(event_log)

In [37]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], utc=True)
df['Year'] = df['time:timestamp'].dt.year
df['Month'] = df['time:timestamp'].dt.month

# Add label

In [38]:
success_log = pm4py.filter_end_activities(event_log, ["A_Pending"])
unsuccess_log = pm4py.filter_end_activities(event_log, ["A_Cancelled", "A_Denied"])
df_suc = pm4py.convert_to_dataframe(success_log)
df_unsuc = pm4py.convert_to_dataframe(unsuccess_log)

df_suc['label'] = 'successful'
df_unsuc['label'] = 'unscucessful'

# Concatenate the DataFrames
df = pd.concat([df_suc, df_unsuc], ignore_index=True)

/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_60174/3298136971.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suc['label'] = 'successful'
/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_60174/3298136971.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unsuc['label'] = 'unscucessful'


In [39]:
df

,Action,EventID,EventOrigin,org:resource,concept:name,lifecycle:transition,time:timestamp,case:LoanGoal,case:concept:name,case:RequestedAmount,case:ApplicationType,Year,Month,label
0,Created,Application_652823628,Application,User_1,A_Create Application,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,Application_652823628,20000.0,New credit,2016,1,successful
1,statechange,ApplState_1582051990,Application,User_1,A_Submitted,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,Application_652823628,20000.0,New credit,2016,1,successful
2,statechange,ApplState_642383566,Application,User_1,A_Concept,complete,2016-01-01 09:52:36.413000+00:00,Existing loan takeover,Application_652823628,20000.0,New credit,2016,1,successful
3,statechange,ApplState_99568828,Application,User_52,A_Accepted,complete,2016-01-02 11:23:04.299000+00:00,Existing loan takeover,Application_652823628,20000.0,New credit,2016,1,successful
4,statechange,ApplState_946455804,Application,User_52,A_Complete,complete,2016-01-02 11:30:28.633000+00:00,Existing loan takeover,Application_652823628,20000.0,New credit,2016,1,successful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130192,statechange,ApplState_660057066,Application,User_12,A_Complete,complete,2017-01-03 19:30:00.773000+00:00,Car,Application_117342811,5000.0,New credit,2017,1,unscucessful
130193,statechange,ApplState_1952793614,Application,User_53,A_Validating,complete,2017-01-11 14:23:09.159000+00:00,Car,Application_117342811,5000.0,New credit,2017,1,unscucessful
130194,statechange,ApplState_1839877280,Application,User_53,A_Incomplete,complete,2017-01-11 14:26:39.541000+00:00,Car,Application_117342811,5000.0,New credit,2017,1,unscucessful
130195,statechange,ApplState_1927619154,Application,User_123,A_Validating,complete,2017-01-13 10:30:50.195000+00:00,Car,Application_117342811,5000.0,New credit,2017,1,unscucessful


# Prefix with full trace before the end event

In [40]:
unique_events = sorted(df['concept:name'].unique())
unique_events

['A_Accepted',
 'A_Cancelled',
 'A_Complete',
 'A_Concept',
 'A_Create Application',
 'A_Denied',
 'A_Incomplete',
 'A_Pending',
 'A_Submitted',
 'A_Validating']

In [41]:
result = pd.crosstab(df['case:concept:name'], df['concept:name'])
result

concept:name,A_Accepted,A_Cancelled,A_Complete,A_Concept,A_Create Application,A_Denied,A_Incomplete,A_Pending,A_Submitted,A_Validating
case:concept:name,,,,,,,,,,
Application_1000158214,1,0,1,1,1,0,0,1,1,1
Application_1000334415,1,0,1,1,1,1,1,0,1,2
Application_100034150,1,0,1,1,1,0,1,1,1,2
Application_1000386745,1,0,1,1,1,0,2,1,1,2
Application_1000474975,1,0,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
Application_998666405,1,1,1,1,1,0,0,0,1,0
Application_999090530,1,0,1,1,1,0,0,1,0,1
Application_999487618,1,0,1,1,1,0,0,1,1,1


In [13]:
result = result.reindex(unique_events, axis=1, fill_value=0)
result

concept:name,A_Accepted,A_Cancelled,A_Complete,A_Concept,A_Create Application,A_Denied,A_Incomplete,A_Pending,A_Submitted,A_Validating
case:concept:name,,,,,,,,,,
Application_1000158214,1,0,1,1,1,0,0,1,1,1
Application_1000334415,1,0,1,1,1,1,1,0,1,2
Application_100034150,1,0,1,1,1,0,1,1,1,2
Application_1000386745,1,0,1,1,1,0,2,1,1,2
Application_1000474975,1,0,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
Application_998666405,1,1,1,1,1,0,0,0,1,0
Application_999090530,1,0,1,1,1,0,0,1,0,1
Application_999487618,1,0,1,1,1,0,0,1,1,1


# Getting trace attribute and merge

In [43]:
case_attr_df = df.groupby('case:concept:name').agg({
    'case:LoanGoal': 'first',
    'case:RequestedAmount': 'first',
    'case:ApplicationType': 'first',
    'Month': 'first',
    'Year': 'first',
    'label': 'first'
}).reset_index()
case_attr_df

,case:concept:name,case:LoanGoal,case:RequestedAmount,case:ApplicationType,Month,Year,label
0,Application_1000158214,Home improvement,12500.0,New credit,6,2016,successful
1,Application_1000334415,"Other, see explanation",5000.0,New credit,9,2016,unscucessful
2,Application_100034150,Existing loan takeover,5000.0,New credit,2,2016,successful
3,Application_1000386745,Car,5000.0,New credit,11,2016,successful
4,Application_1000474975,"Other, see explanation",50000.0,New credit,6,2016,successful
...,...,...,...,...,...,...,...
17544,Application_998666405,Not speficied,45000.0,New credit,10,2016,unscucessful
17545,Application_999090530,Car,7500.0,New credit,8,2016,successful
17546,Application_999487618,Existing loan takeover,20000.0,New credit,8,2016,successful
17547,Application_999507989,Not speficied,24000.0,New credit,3,2016,successful


In [45]:
merged_df = pd.merge(result, case_attr_df, on='case:concept:name', how='inner')
merged_df

,case:concept:name,A_Accepted,A_Cancelled,A_Complete,A_Concept,A_Create Application,A_Denied,A_Incomplete,A_Pending,A_Submitted,A_Validating,case:LoanGoal,case:RequestedAmount,case:ApplicationType,Month,Year,label
0,Application_1000158214,1,0,1,1,1,0,0,1,1,1,Home improvement,12500.0,New credit,6,2016,successful
1,Application_1000334415,1,0,1,1,1,1,1,0,1,2,"Other, see explanation",5000.0,New credit,9,2016,unscucessful
2,Application_100034150,1,0,1,1,1,0,1,1,1,2,Existing loan takeover,5000.0,New credit,2,2016,successful
3,Application_1000386745,1,0,1,1,1,0,2,1,1,2,Car,5000.0,New credit,11,2016,successful
4,Application_1000474975,1,0,1,1,1,0,0,1,0,1,"Other, see explanation",50000.0,New credit,6,2016,successful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17544,Application_998666405,1,1,1,1,1,0,0,0,1,0,Not speficied,45000.0,New credit,10,2016,unscucessful
17545,Application_999090530,1,0,1,1,1,0,0,1,0,1,Car,7500.0,New credit,8,2016,successful
17546,Application_999487618,1,0,1,1,1,0,0,1,1,1,Existing loan takeover,20000.0,New credit,8,2016,successful
17547,Application_999507989,1,0,1,1,1,0,1,1,1,1,Not speficied,24000.0,New credit,3,2016,successful
